# Test dataset building and analysis for DATA 698 proposal
## 1000 papers example

In [66]:
import os
import ads as ads 
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
os.environ["ADS_DEV_KEY"] = "kNUoTurJ5TXV9hsw9KQN1k8wH4U0D7Oy0CJoOvyw"

In [3]:
ads.config.token = 'ADS_DEV_KEY' 

In [19]:
#Search for papers (1000 most cited)
papers1 = list(ads.SearchQuery(q= "*", sort="citation_count", max_pages=20 ))

In [20]:
# find titles 
t = []
for i in papers1:
    title1 = i.title
    t.append(title1)
title = t

//anaconda/lib/python3.5/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'title', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [21]:
# find abstracts 
a = []
for i in papers1:
    abstract1 = i.abstract
    a.append(abstract1)
abstract = a

//anaconda/lib/python3.5/site-packages/ads/utils.py:23: UserWarning: You are lazy loading attributes via 'abstract', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [22]:
# create an initial df
df = pd.DataFrame({'Title' : title,
 'Abstract':abstract
  })

In [23]:
df['Title'] = df['Title'].str.get(0)

In [26]:
df.to_csv("top1000.csv", sep=',', encoding='utf-8')

In [33]:
# average number of words in abstract
word_count_abstact = df['Abstract'].str.split().str.len()
word_count_abstact.mean()

152.98317307692307

In [34]:
# average number of words in title
word_count_title = df['Title'].str.split().str.len()
word_count_title.mean()

8.159

#### Using TFIDF, cosine similarity, and a vector space model to compare first title to all other titles

In [42]:
tfidf_vectorizer = TfidfVectorizer() #term frequency inverse document frequency

In [46]:
# convert title column to tuple
title_tuple = tuple(list(df['Title']))

In [49]:
# create TFIDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(title_tuple)
tfidf_matrix.shape # 2273 tfidf terms (the number of columns from the matrix)

(1000, 2273)

In [89]:
# calculate cosine similarity - first title compared to all others
# The tfidf_matrix[0:1] is the Scipy operation to get the first row of the sparse matrix and 
# the resulting array is the Cosine Similarity between the first title with all titles in the set.
# Note: first value of the array is 1.0 bc it is the Cosine Similarity between the first title with itself.

# print full array
cosine_sim_array = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)
cosine_sim_array

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.14694283,  0.        ,  0.        ,
         0.        ,  0.16915831,  0.        ,  0.        ,  0.14740682,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.42891328,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [90]:
# Show most similar title in degrees
cos_sim = np.partition(cosine_sim_array.flatten(), -2)[-2] # second largest value in array (most similar title)
angle_in_degrees = math.acos(cos_sim)
math.degrees(angle_in_degrees)

51.7815786510802

In [112]:
# find index of most similar title
itemindex = np.where(cosine_sim_array==cos_sim)
itemindex = itemindex[1]
itemindex = itemindex[0]
itemindex

279

In [120]:
# find most similar title and print it along with first title used for comparison
comparison_title = title_tuple[0]
most_similar_title = title_tuple[itemindex]

In [121]:
comparison_title

'Generalized Gradient Approximation Made Simple'

In [122]:
most_similar_title

'Generalized Gradient Approximation Made Simple [Phys. Rev. Lett. 77, 3865 (1996)]'

## Weird! These are actually two separate records with different publication dates??
### Works though...